# 08_04: Comparing quantitative variables

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
gm = pd.read_csv('gapminder.csv')

In [ ]:
gm.info()

In [ ]:
italy = gm.query('country == "Italy"')

In [ ]:
italy.plot.scatter(x='year', y='population', s=3, figsize=(4.5,3));

In [ ]:
axes = gm.query('country == "China"').plot.scatter(x='year', y='population', label='china', c='C0', s=3, figsize=(4.5,3))
gm.query('country == "India"').plot.scatter(x='year', y='population', label='india', c='C1', s=3, ax=axes); # reuse axes

In [ ]:
italy.plot.scatter('year', 'gdp_per_capita', logy=True, s=3, figsize=(4.5,3));

In [ ]:
italy.plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, s=3, figsize=(4.5,3));

In [ ]:
italy.plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, c=italy.year,
                   colormap='viridis', s=5, figsize=(5.5,3));

In [ ]:
italy.plot.scatter('year', 'life_expectancy', logy=True, s=3, figsize=(4,3));

In [ ]:
gm2015 = gm[gm.year == 2015]

In [ ]:
gm2015.plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, s=1e-6 * gm2015.population);

In [ ]:
gm2015.plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, s=1e-6 * gm2015.population,
                    edgecolor='w');

In [ ]:
continent = gm2015.region.map({'Africa': 'skyblue', 'Europe': 'gold', 'America': 'palegreen', 'Asia': 'coral', 'Oceania': 'teal'})
continent

In [ ]:
gm2015.plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, s=1e-6 * gm2015.population,
                    c=continent, edgecolor='gray');

In [ ]:
gm2015.plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, s=1e-6 * gm2015.population,
                    c=continent, edgecolor=np.where(gm2015.country == "USA", 'black', 'gray'));

In [ ]:
def plotyear(year, country):
    gm[gm.year == year].plot.scatter(x='gdp_per_capita', y='life_expectancy', logx=True, s=1e-6 * gm2015.population,
                                     c=continent, edgecolor=np.where(gm2015.country == country, 'black', 'gray'));
    pp.axis(xmin=200, xmax=200000, ymin=15, ymax=90); # fix the plotting ranges

In [ ]:
from ipywidgets import interactive, widgets

In [ ]:
interactive_plot = interactive(plotyear,
                               year=widgets.IntSlider(min=1950, max=2023, step=1, value=1950),
                               country=widgets.Dropdown(options=['USA', 'China', 'India']))

# some versions of Jupyter require workaround to keep the figure from jumping around
# interactive_plot.children[-1].layout.height = '400px'
interactive_plot